### Notebook for workings on the DE1 tables

In [ ]:
library(haven)
library(PxWebApiData)
suppressPackageStartupMessages(library(tidyverse))
library(klassR)

library(httr)
library(jsonlite)

library(glue)

In [ ]:
source("Functions.r")

#### Statbank approach:

In [ ]:
# import data

df_DE1 <- ApiData(07459,
              Alder = list("agg:FemAarigGruppering",
                           c("F00-04", "F05-09", "F10-14",
                             "F15-19", "F20-24", "F25-29",
                             "F30-34", "F35-39", "F40-44",
                             "F45-49", "F50-54", "F55-59",
                             "F60-64", "F65-69", "F70-74",
                             "F75-79", "F80-84", "F85-89",
                             "F90-94", "F95-99", "F100G5+")),
              Region = c("0301", "1103", "4601", "5001"),
              Tid = "2022")[[2]]

In [ ]:
# import klass correspondence for city names

df_DE1$navn <- ApplyKlass(df_DE1$Region, klass = 131)

city_correspondence <- GetKlass(550) %>%
                    select(code, name)

In [ ]:
# merge klass correspondence with dataframe

df_DE1 <- merge.data.frame(df_DE1, city_correspondence,
                           by.x = "navn",
                           by.y = "name")

In [ ]:
df_DE1

### Filter + sum rows (TOTAL TABELLER)

In [ ]:
DE1040V_2022 <- sum_rows(Alder, "F00-04")
DE1040V_2022

### Filter (KJØNNSFORDELTE TABELLER)

In [ ]:
df_DE1 %>%
    filter(df_DE1["Alder"] == "F00-04",
           df_DE1["Kjonn"] == "1") %>%
    select(code, value) %>%
    rename(Value = value)

In [ ]:
sum_rows_2("F05-09", "1")

In [ ]:
DE1074V_2022 <- df_DE1 %>%
                    filter(Alder == "F05-09") %>%
                    group_by(code) %>%
                    summarise(Value = sum(value))

In [ ]:
# sum rows

DE1001V_2022 <- df_DE1 %>%
                   group_by(code) %>%
                   summarise(Value = sum(value))

In [ ]:
DE1001V_2022

In [ ]:
table_formatting(DE1001V_2022)

In [ ]:
# sum rows (men)

DE1002V_2022 <- df_DE1 %>%
                   filter(Kjonn == 1) %>%
                   group_by(code) %>%
                   summarise(Value = sum(value))

In [ ]:
DE1002V_2022

In [ ]:
# sum rows (women)

DE1003V_2022 <- df_DE1 %>%
                   filter(Kjonn == 2) %>%
                   group_by(code) %>%
                   summarise(Value = sum(value))

In [ ]:
DE1003V_2022

In [ ]:
# Loop: for each age group level, produce three tables (all, men, women). 